In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
model=ChatGoogleGenerativeAI(model='gemini-2.5-flash')
parser=StrOutputParser()

In [3]:
result=model.invoke('hy')

In [4]:
result

AIMessage(content='Hey there! How can I help you today? 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--43772192-cc68-4d02-b584-752acadf4401-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}})

In [5]:
result.content

'Hey there! How can I help you today? 😊'

In [6]:
parser.invoke(result)

'Hey there! How can I help you today? 😊'

In [7]:
while True: 
    message=input("Write your query")
    if message=='end': 
        print('Good by have a great day')
        break
    else: 
        print(message, ":", parser.invoke(model.invoke([HumanMessage(content=message)])))


# here it is not keeping the track of memory so it is not a chat bot basicaly it is Q&a System without tracking the memory

hy : Hey there! How can I help you today? 😊
my name is pankaj : Hello Pankaj! Nice to meet you.
what is my name : I don't know your name. As an AI, I don't have access to personal information about you unless you choose to tell me.

If you'd like to tell me your name, I'd be happy to use it in our chat!
Good by have a great day


## 1 chat history manualy


In [ ]:

chat_history=[AIMessage(content="You are a helpful assistant")]

while True: 
    message=input('Write your query')
    chat_history.append(HumanMessage(content=message))
    if message=='end': 
        print('Good by have a great day')
        break
    else: 
        response=parser.invoke(model.invoke(chat_history))
        chat_history.append(AIMessage(content=response))
        print(f'HumanMessage: {message}\nAIMessage: {response}')



# this is the manual way to store chat history

HumanMessage: hy
AIMessage: Hello! How can I help you today?
HumanMessage: my name is pankaj kumawat
AIMessage: Nice to meet you, Pankaj! How can I help you today?
HumanMessage: what is the capital of india
AIMessage: The capital of India is **New Delhi**.
HumanMessage: what will the result of 2 + 2
AIMessage: The result of 2 + 2 is **4**.
HumanMessage: what is my name
AIMessage: Your name is **Pankaj Kumawat**.
HumanMessage: what was the last math operation
AIMessage: The last math operation we discussed was **addition** (specifically, 2 + 2).
HumanMessage: what is the population of the previouos city
AIMessage: The previous city we discussed was **New Delhi**.

The population of the **National Capital Territory (NCT) of Delhi**, which includes New Delhi, is estimated to be around **30 million** people (as of recent estimates for the broader metropolitan area).
Good by have a great day


## 2 Chat history using Predefined functions

In [8]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

| Component                        | Purpose                                                | Use                                                       |
| -------------------------------- | ------------------------------------------------------ | --------------------------------------------------------- |
| **`BaseChatMessageHistory`**     | Blueprint (interface) for storing chat history         | Used when creating a **custom** memory backend (like DB). |
| **`InMemoryChatMessageHistory`** | Stores chat history **temporarily in RAM**             | Used for simple or short-lived sessions.                  |
| **`RunnableWithMessageHistory`** | Wraps your chain/model to **auto-manage chat history** | Lets the model remember previous conversations.           |


👉 In short:<br>
BaseChatMessageHistory = rulebook<br>
InMemoryChatMessageHistory = ready-to-use memory<br>
RunnableWithMessageHistory = makes your chatbot remember chats <br>automatically ✅

In [11]:
store={}
def get_session_history(session_id: str) -> BaseChatMessageHistory: 
    if session_id not in store: 
        store[session_id]=InMemoryChatMessageHistory()
    return store[session_id]

In [12]:
model_with_memory=RunnableWithMessageHistory(model, get_session_history)

In [13]:
config={'configurable': {'session_id': 'firstchat'}}
model_with_memory.invoke([HumanMessage(content='hy i am pankaj kumawat')], config=config).content

'Hi Pankaj! How can I help you today?'

In [14]:
model_with_memory.invoke([HumanMessage(content='what is my name')], config=config).content

'Based on what you told me, your name is Pankaj Kumawat.'

                           ┌──────────────────────────────────────────────┐
                           │ 1️⃣ You call invoke()                         │
                           │ model_with_memory.invoke(                    │
                           │   [HumanMessage('hy i am pankaj kumawat')],  │
                           │   config={'configurable':{'session_id':'firstchat'}} │
                           └──────────────────────────────────────────────┘
                                                │
                                                ▼
                 ┌──────────────────────────────────────────────────────────┐
                 │ 2️⃣ Extract session_id from config                       │
                 │ session_id = config['configurable']['session_id']        │
                 │ → 'firstchat'                                            │
                 └──────────────────────────────────────────────────────────┘
                                                │
                                                ▼
                 ┌──────────────────────────────────────────────────────────┐
                 │ 3️⃣ Call get_session_history(session_id)                 │
                 │ - Checks if 'firstchat' exists in store                  │
                 │ - If not → creates InMemoryChatMessageHistory()          │
                 │ - Returns chat history object                            │
                 └──────────────────────────────────────────────────────────┘
                                                │
                                                ▼
                 ┌──────────────────────────────────────────────────────────┐
                 │ 4️⃣ Retrieve previous messages (if any)                  │
                 │ past_messages = history.messages                         │
                 │ (Empty if this is the first message)                     │
                 └──────────────────────────────────────────────────────────┘
                                                │
                                                ▼
                 ┌──────────────────────────────────────────────────────────┐
                 │ 5️⃣ Append new HumanMessage                              │
                 │ combined_input = past_messages + [new HumanMessage]      │
                 └──────────────────────────────────────────────────────────┘
                                                │
                                                ▼
                 ┌──────────────────────────────────────────────────────────┐
                 │ 6️⃣ Pass combined messages to model.invoke()              │
                 │ model processes conversation and generates reply         │
                 │ → returns AIMessage(content="Hello Pankaj!")             │
                 └──────────────────────────────────────────────────────────┘
                                                │
                                                ▼
                 ┌──────────────────────────────────────────────────────────┐
                 │ 7️⃣ Update memory                                         │
                 │ history.add_user_message("hy i am pankaj kumawat")       │
                 │ history.add_ai_message("Hello Pankaj!")                  │
                 └──────────────────────────────────────────────────────────┘
                                                │
                                                ▼
                 ┌──────────────────────────────────────────────────────────┐
                 │ 8️⃣ Store updated history back into `store`              │
                 │ store['firstchat'] = updated InMemoryChatMessageHistory  │
                 └──────────────────────────────────────────────────────────┘
                                                │
                                                ▼
                 ┌──────────────────────────────────────────────────────────┐
                 │ 9️⃣ Return AIMessage.content                             │
                 │ "Hello Pankaj! How can I help you today?"                │
                 └──────────────────────────────────────────────────────────┘



In [10]:

# After 1st Run — Your Memory Looks Like This
# store = {
#   "firstchat": InMemoryChatMessageHistory(
#       messages=[
#           HumanMessage(content="hy i am pankaj kumawat"),
#           AIMessage(content="Hello Pankaj! How can I help you today?")
#       ]
#   )
# }

In [16]:
config={'configurable': {'session_id': 'firstchat'}}
model_with_memory.invoke([HumanMessage(content='what is my name?')], config=config).content

'Your name is Pankaj Kumawat.'

In [19]:
# Flow repeats, but this time step 3 finds existing memory, so the model sees the full context:

# Human: hy i am pankaj kumawat
# AI: Hello Pankaj! How can I help you today?
# Human: what is my name?


In [17]:
config = {"configurable": {"session_id": "secondtchat"}}
model_with_memory.invoke([HumanMessage(content='what is my name')], config=config).content

"I still don't know your name. As an AI, I don't have personal memory of past conversations or access to any of your private information. I don't know who you are.\n\nIf you want me to call you something, please let me know!"

In [21]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='hy i am pankaj kumawat', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi Pankaj! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--fbf83d7b-ab1b-4126-b43e-757385f738d8-0', usage_metadata={'input_tokens': 8, 'output_tokens': 11, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='Based on what you told me, your name is Pankaj Kumawat.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--f3e90796-b1e8-41f0-8b04-ef9edae5c4c2-0', usage_metadata={'input_tokens': 25, 'output_tokens': 15, 'total_tokens': 159, 'input_token_details': {'cache_read':

In [22]:
config={'configurable': {'session_id': 'third'}}

while True: 
    message=input('Write your query')
    if message=='end': 
        print('Have a great Day')
        break
    response=model_with_memory.invoke([HumanMessage(content=message)], config=config).content
    print(f'HumanMessage: {message}\nAIMessage: {response}')

HumanMessage: hy
AIMessage: Hi there! How can I help you today?
HumanMessage: my name is pankaj
AIMessage: Nice to meet you, Pankaj! Is there anything I can help you with?
HumanMessage: what is the capital of india
AIMessage: The capital of India is **New Delhi**.
HumanMessage: what is my name
AIMessage: Your name is Pankaj.
HumanMessage: what is the population of that city
AIMessage: The population of New Delhi itself is relatively small as it's a specific district within the larger National Capital Territory (NCT) of Delhi.

However, if you're asking about the **National Capital Territory of Delhi (NCT of Delhi)**, which is the broader metropolitan area that includes New Delhi, its population is significantly larger.

According to various estimates, the population of the **NCT of Delhi** is around **30-33 million people** as of 2023-2024. This makes it one of the largest metropolitan areas in the world.

Keep in mind that population figures are always estimates and can vary slightly 

In [23]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='hy i am pankaj kumawat', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi Pankaj! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--fbf83d7b-ab1b-4126-b43e-757385f738d8-0', usage_metadata={'input_tokens': 8, 'output_tokens': 11, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='Based on what you told me, your name is Pankaj Kumawat.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--f3e90796-b1e8-41f0-8b04-ef9edae5c4c2-0', usage_metadata={'input_tokens': 25, 'output_tokens': 15, 'total_tokens': 159, 'input_token_details': {'cache_read':

In [24]:
store['third'].messages

[HumanMessage(content='hy', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--646c58df-0237-4481-a951-d7402b77e8de-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content='my name is pankaj', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice to meet you, Pankaj! Is there anything I can help you with?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--c0510609-7cf3-42c4-a8c7-4c4bb354580c-0', usage_metadata={'input_tokens': 19, 'output_tokens': 17, 'total_tokens': 81, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content='what is the capital of india

## same thing using Chains

In [25]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [27]:

prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)

In [28]:
chain=prompt | model | parser

In [31]:
chain.invoke({'messages': [HumanMessage(content='Hi i am pankaj')]})

"Hi Pankaj! It's nice to meet you.\n\nHow can I help you today?"

In [32]:
chain.invoke({'messages': [HumanMessage(content='what is my name')]})

"As an AI, I don't have access to personal information about you, including your name. I don't store any details about our conversations or who you are."

In [33]:
model_with_memory=RunnableWithMessageHistory(chain, get_session_history)

In [34]:
config={'configurable': {'session_id': 'fourthchat'}}

In [38]:
model_with_memory.invoke([HumanMessage(content='hy i am chuha')], config=config)

'Okay, so you\'re Chuha now! I\'m keeping track. From "Raja ka Bacha" to "Chuha" – quite a range!\n\nIt\'s good to meet you, Chuha! How can I help you today?'

In [39]:
model_with_memory.invoke([HumanMessage(content='what is my name')], config=config)

'Based on our last interaction, the nickname you just gave me is **Chuha**.'

In [40]:
model_with_memory.invoke([HumanMessage(content='what will be the output of 4*8')], config=config)

'The output of 4 * 8 is **32**.'

In [41]:
model_with_memory.invoke([HumanMessage(content='what maths operation done recently?')], config=config)

'The mathematical operation done recently was **multiplication** (4 * 8).'

In [43]:
config={'configurable': {'session_id': 'fifthchat'}}
while True: 
    message=input('Enter your query: ')
    if message=='end': 
        print("have a great day! ")
        break
    response=model_with_memory.invoke([HumanMessage(content=message)], config=config)
    print(f'HumanMessage: {message}\nAIMessage: {response}')

HumanMessage: hy
AIMessage: Hi there! How can I help you today?
HumanMessage: my name is pankaj kumar kumhar
AIMessage: Nice to meet you, Pankaj Kumar Kumhar!

How can I help you today, Pankaj?
HumanMessage: what is the output of 9*9
AIMessage: The output of 9 * 9 is **81**.
HumanMessage: what is my name
AIMessage: Your name is **Pankaj Kumar Kumhar**.
HumanMessage: what maths operations done recently add them to 9
AIMessage: The math operation done recently was multiplication: **9 * 9 = 81**.

Now, adding that result to 9:
81 + 9 = **90**
have a great day! 


In [44]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='hy i am pankaj kumawat', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi Pankaj! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--fbf83d7b-ab1b-4126-b43e-757385f738d8-0', usage_metadata={'input_tokens': 8, 'output_tokens': 11, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='Based on what you told me, your name is Pankaj Kumawat.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--f3e90796-b1e8-41f0-8b04-ef9edae5c4c2-0', usage_metadata={'input_tokens': 25, 'output_tokens': 15, 'total_tokens': 159, 'input_token_details': {'cache_read':

In [46]:
store['fifthchat'].messages

[HumanMessage(content='hy', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='my name is pankaj kumar kumhar', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice to meet you, Pankaj Kumar Kumhar!\n\nHow can I help you today, Pankaj?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the output of 9*9', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The output of 9 * 9 is **81**.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is **Pankaj Kumar Kumhar**.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what maths operations done recently add them to 9', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The math operation done recently was multiplication: **

## Trimming the conversation

In [47]:
from langchain_core.messages import SystemMessage, trim_messages

In [53]:
trimmer=trim_messages(
    max_tokens=40, 
    strategy='last', 
    token_counter=model, 
    include_system=True, 
    allow_partial=False, 
    start_on='human'
)

In [54]:


messages = [
    HumanMessage(content="hi! I'm pankaj"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [55]:
model.get_num_tokens_from_messages(messages)

52

In [57]:
trimmed_message=trimmer.invoke(messages)

In [58]:
trimmed_message

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [59]:
model.get_num_tokens_from_messages(trimmed_message)

39

In [61]:

prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)

In [62]:
prompt

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [63]:
messages

[HumanMessage(content="hi! I'm pankaj", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [64]:
trimmed_message

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [65]:
trimmed_message + [HumanMessage(content='what is my name')]

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={})]

In [66]:
messages+[HumanMessage(content="what's my name?")]


[HumanMessage(content="hi! I'm pankaj", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={})]

In [69]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter('messages') | trimmer)
    | prompt
    | model
)


In [70]:
response= chain.invoke(
    {
        'messages': messages + [HumanMessage(content='what is my name?')], 
        'language': 'English'
    }
)

print(response.content)

I don't know your name. As an AI, I don't have access to personal information about you unless you tell me.


In [71]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2".'

In [72]:
model_with_memory=RunnableWithMessageHistory(
    chain, 
    get_session_history, 
    input_messages_key='messages'
)

In [ ]:
config = {"configurable": {"session_id": "sixthchat"}}


In [74]:


response = model_with_memory.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I don't know your name! As an AI, I don't have access to personal information about you unless you choose to tell me."

In [75]:


response = model_with_memory.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't asked me a math problem yet!\n\nIf you have one, please feel free to share it, and I'll do my best to help you solve it or understand it."

In [78]:
config = {"configurable": {"session_id": "sixthchat"}}

while True: 
    message=input('Enter your query')
    if message=='end': 
        print('Have a great day')
        break
    response = model_with_memory.invoke(
    {
        "messages": [HumanMessage(content=message)],
        "language": "English",
    },
    config=config,
    )

    print(f'HumanMessage: {message}\nAIMessage: {response.content}')
    


HumanMessage: hy my name is pankaj
AIMessage: Hi Pankaj! It's nice to meet you.

How can I help you today?
HumanMessage: what is my name
AIMessage: Your name is Pankaj!
HumanMessage: i like to traveling
AIMessage: That's wonderful! Traveling is such a fantastic way to experience new cultures, see beautiful places, and create lasting memories.

What kind of travel do you enjoy most? Do you have a favorite destination you've visited, or somewhere you dream of going?
HumanMessage: what is my name
AIMessage: I am an AI and I don't have access to personal information about you, so I don't know your name. You haven't told me what it is!
Have a great day


In [79]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='hy i am pankaj kumawat', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi Pankaj! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--fbf83d7b-ab1b-4126-b43e-757385f738d8-0', usage_metadata={'input_tokens': 8, 'output_tokens': 11, 'total_tokens': 19, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='Based on what you told me, your name is Pankaj Kumawat.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--f3e90796-b1e8-41f0-8b04-ef9edae5c4c2-0', usage_metadata={'input_tokens': 25, 'output_tokens': 15, 'total_tokens': 159, 'input_token_details': {'cache_read':

In [81]:
store['sixthchat'].messages

[HumanMessage(content='hy', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--bb3b1782-d32c-444e-8d46-ca333168dd27-0', usage_metadata={'input_tokens': 18, 'output_tokens': 9, 'total_tokens': 67, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content='my name is pankaj', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice to meet you, Pankaj! Is there anything I can help you with?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--c639cbc4-a088-466b-976a-0d3780dc7970-0', usage_metadata={'input_tokens': 34, 'output_tokens': 17, 'total_tokens': 91, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content='what is the output of 8+9', add